In [43]:
import numpy as np
from math import log

In [95]:
# init
datapoints = [(4,100),(2,99),(1,97),(0.5,83.4),(0.25,42),(0.125,10),(0.062,3.2),(0.031,2)]

dtype = [('diameters', float), ('fractions', float)]
datapts = np.array(datapoints, dtype=dtype)       # create a structured array

# scale
if max(datapts['fractions']) > 1.:
    datapts['fractions'] = datapts['fractions'] / 100.

datapts = np.sort(datapts, order='diameters')       

psi_scale = np.array(range(-10,20), dtype = np.float)
psi_scale_mm = 2**psi_scale

if datapts['fractions'][0] > 0.0:
    # if smallest fraction is not zero, assume that the next smallest psi scale grain size had a fraction of zero
    next_smallest_size = psi_scale_mm[np.where(psi_scale_mm<datapts['diameters'][0])[0][-1]]
    first_entry = np.array([(next_smallest_size,0.0)], dtype = dtype)
    datapts = np.hstack((first_entry,datapts))
    
if datapts['fractions'][-1] < 1.0:
    # if largest fraction is not zero, assume that the next larger psi scale grain size had a fraction of 1
    next_larger_size = psi_scale_mm[np.where(psi_scale_mm>datapts['diameters'][-1])[0][0]]
    last_entry = np.array([(next_larger_size,1.0)], dtype = dtype)
    datapts = np.hstack((datapts,last_entry))

In [ ]:
# run

# geometric mean



# geometric standard deviation